In [29]:
import polars as pl
import turbofats
from tqdm import tqdm
import os
from scripts.utils import read_light_curves
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
with open("dataset/4clasess.pkl","rb") as f: ##se carga el dataset con mas de 20 observaciones,sin nans,-inf,inf 
    data = pkl.load(f)

In [60]:
feature_list = [
    "PeriodLS_v2",
    "Period_fit_v2",
    "Amplitude",
    "Rcs",
    "StetsonK",
    "Meanvariance",
    "Autocor_length",
    "SlottedA_length",
    "StetsonK_AC",
    #"StetsonL",
    "Con",
    #"Color",
    "Beyond1Std",
    "SmallKurtosis",
    "Std",
    "Skew",
    #"StetsonJ",
    "MaxSlope",
    "MedianAbsDev",
    "MedianBRP",
    "PairSlopeTrend",
    "FluxPercentileRatioMid20",
    "FluxPercentileRatioMid35",
    "FluxPercentileRatioMid50",
    "FluxPercentileRatioMid65",
    "FluxPercentileRatioMid80",
    "PercentDifferenceFluxPercentile",
    "PercentAmplitude",
    "LinearTrend",
    #"Eta_color",
    "Eta_e",
    "Mean",
    "Q31",
    #"Q31_color",
    "AndersonDarling",
    "Psi_CS_v2",
    "Psi_eta_v2",
    "Gskew",
    "StructureFunction_index_21",
    "StructureFunction_index_31",
    "StructureFunction_index_32",
    "Pvar",
    "ExcessVar",
    "PeriodPowerRate",
    "IAR_phi",
    "CIAR_phiR_beta",
    "SF_ML_amplitude",
    "SF_ML_gamma",
    "GP_DRW_sigma",
    "GP_DRW_tau",
    #"Harmonics"
]


fs = turbofats.FeatureSpace(feature_list=feature_list)

In [61]:
df = pd.DataFrame({
    "time": data[102]['mjd'],
    "magnitude": data[102]['flux'],
    "error": data[102]['err']
})
df['id'] = 0
df.set_index('id', inplace=True)
values = fs.calculate_features(lightcurve=df)
values

,PeriodLS_v2,Period_fit_v2,Amplitude,Rcs,StetsonK,Meanvariance,Autocor_length,SlottedA_length,StetsonK_AC,Con,...,StructureFunction_index_32,Pvar,ExcessVar,PeriodPowerRate,IAR_phi,CIAR_phiR_beta,SF_ML_amplitude,SF_ML_gamma,GP_DRW_sigma,GP_DRW_tau
oid,,,,,,,,,,,,,,,,,,,,,
0,0.084485,0.000005,1.632005,0.216081,0.856104,-6.905519e+15,1.0,0.003183,0.782268,0.0,...,1.28482,1.0,4.222068e+31,0.021593,4.230063e-13,-2.167605e-09,-0.5,-0.5,0.815474,0.00049


In [62]:
import polars as pl
import turbofats
from tqdm import tqdm
import os
from scripts.utils import read_light_curves
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

with open("dataset/4clasess.pkl","rb") as f:
    data = pkl.load(f)

feature_list = [
    "PeriodLS_v2",
    "Period_fit_v2",
    "Amplitude",
    "Rcs",
    "StetsonK",
    "Meanvariance",
    "Autocor_length",
    "SlottedA_length",
    "StetsonK_AC",
    #"StetsonL",
    "Con",
    #"Color",
    "Beyond1Std",
    "SmallKurtosis",
    "Std",
    "Skew",
    #"StetsonJ",
    "MaxSlope",
    "MedianAbsDev",
    "MedianBRP",
    "PairSlopeTrend",
    "FluxPercentileRatioMid20",
    "FluxPercentileRatioMid35",
    "FluxPercentileRatioMid50",
    "FluxPercentileRatioMid65",
    "FluxPercentileRatioMid80",
    "PercentDifferenceFluxPercentile",
    "PercentAmplitude",
    "LinearTrend",
    #"Eta_color",
    "Eta_e",
    "Mean",
    "Q31",
    #"Q31_color",
    "AndersonDarling",
    "Psi_CS_v2",
    "Psi_eta_v2",
    "Gskew",
    "StructureFunction_index_21",
    "StructureFunction_index_31",
    "StructureFunction_index_32",
    "Pvar",
    "ExcessVar",
    "PeriodPowerRate",
    "IAR_phi",
    "CIAR_phiR_beta",
    "SF_ML_amplitude",
    "SF_ML_gamma",
    "GP_DRW_sigma",
    "GP_DRW_tau",
    #"Harmonics"
]


fs = turbofats.FeatureSpace(feature_list=feature_list)

all_features = []

for i, lc in tqdm(enumerate(data), total=len(data)):
    df = pd.DataFrame({
        "time": lc['mjd'],
        "magnitude": lc['flux'],
        "error": lc['err']
    })
    df['id'] = i
    df.set_index('id', inplace=True)

    try:
        features = fs.calculate_features(lightcurve=df)
        features['id'] = i
        all_features.append(features)
    except Exception as e:
        print(f"Error en curva {i}: {e}")


final_df = pd.concat(all_features, ignore_index=True)

print(final_df.head())


  0%|          | 1/178754 [00:00<35:55:25,  1.38it/s] /home/llanking/tesis/turbo-fats/turbofats/FeatureFunctionLib.py:81: RuntimeWarning:divide by zero encountered in scalar divide
 /home/llanking/tesis/turbo-fats/turbofats/FeatureFunctionLib.py:905: RuntimeWarning:divide by zero encountered in scalar divide
  0%|          | 11/178754 [00:16<52:42:24,  1.06s/it] /home/llanking/tesis/turbo-fats/turbofats/FeatureSpace.py:59: UserWarning:Exception when computing turbo-fats feature: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
 /home/llanking/micromamba/envs/turbo-fats/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:Mean of empty slice.
 /home/llanking/micromamba/envs/turbo-fats/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning:invalid value encountered in scalar divide
 /home/llanking/micromamba/envs/turbo-fats/lib/python3.8/site-packages/numpy/core/_methods.py:269: Run

KeyboardInterrupt: 

## Cosas de la charla
- Revisar el titulo del trabajo de titulo
- Se entendió bien el problema y porqué es necesario hacer pruebas :) 
- Discusión sobre el titulo.